<a href="https://colab.research.google.com/github/RichardLin3/InfoSysProj/blob/main/ISProj_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import os

repo_url = "https://github.com/RichardLin3/InfoSysProj.git"
repo_dir = "InfoSysProj"

if os.path.exists(repo_dir) and os.listdir(repo_dir):
    print(f"Directory '{repo_dir}' already exists and is not empty. Skipping git clone.")
else:
    print(f"Cloning '{repo_url}' into '{repo_dir}'...")
    !git clone {repo_url}

Directory 'InfoSysProj' already exists and is not empty. Skipping git clone.


In [21]:
import json

with open('InfoSysProj/data/minors_v2.json', 'r') as f:
    minor_data = json.load(f)

with open('InfoSysProj/data/gened_v2.json', 'r') as f:
    gened_data = json.load(f)

with open('InfoSysProj/data/major_v2.json', 'r') as f:
    major_data = json.load(f)

with open('InfoSysProj/data/liberal_arts_courses.json', 'r') as f:
    lib_data = json.load(f)

with open('InfoSysProj/data/ID.json', 'r') as f:
    ID_data = json.load(f)

In [22]:
major_names = list(major_data.keys())
minor_names = list(minor_data.keys())

In [23]:
#major selection
import ipywidgets as widgets
from IPython.display import display

dropdown = widgets.Dropdown(
    options=major_names,
    description='Select a Major:',
    disabled=False,
)

def on_change(change):
    global selected_major
    if change['type'] == 'change' and change['name'] == 'value':
        selected_major = change['new']
        print(f"You selected: {selected_major}")

dropdown.observe(on_change)

display(dropdown)

Dropdown(description='Select a Major:', options=('Data Science BS', 'Applied Chemistry BS', 'Architecture Tech…

You selected: Architecture BARCH


In [24]:
dropdown2 = widgets.Dropdown(
    options=minor_names,
    description='Select a Minor:',
    disabled=False,
)

def on_change2(change):
    global selected_minor
    if change['type'] == 'change' and change['name'] == 'value':
        selected_minor = change['new']
        print(f"You selected: {selected_minor}")

dropdown2.observe(on_change2)

display(dropdown2)

Dropdown(description='Select a Minor:', options=('Arabic Language and Cultural Studies', 'Art History & Visual…

You selected: Creative Writing


In [42]:
#checks if a major/minor is selected otherwise default to the first
if selected_major is None:
  selected_major = major_names[0]
if minor_names is None:
  selected_minor = minor_names[0]

In [43]:
courses = []  # Initialize an empty list to store course codes

if selected_minor in minor_data.keys():  # Check if the selected minor exists in minor_data
    minor_info = minor_data[selected_minor]
    for section_name, section_list in minor_info.items(): # Iterate through the items of the minor_info dictionary
        # section list is usually a list
        for entry in section_list:
            # Check if the entry is a dictionary and has a 'Course' key
            if isinstance(entry, dict) and 'Course' in entry:
                courses.append(entry['Course'])
            # Check if the entry is a dictionary and contains nested course options (like 'Choose 1')
            elif isinstance(entry, dict):
                for key, value in entry.items():
                    if isinstance(value, list):
                        for item in value:
                            if isinstance(item, dict) and 'Course' in item:
                                courses.append(item['Course'])


print(courses) # Print the extracted courses to verify

['ENG 1141', 'ENG 2142', 'ENG 2143', 'ENG 2144', 'ENG 2145', 'ENG 1151', 'ENG 2001', 'ENG 2002', 'ENG 2003', 'ENG 2420', 'ENG 2420ID', 'ENG 2720', 'ENG 3402', 'ENG 3403', 'ENG 3760']


^^ it checks if we found the right minor, then goes inside its “Requirements” and collects every course code.
if there’s a “choose 1” section, it digs into that too and adds those course codes to the list.

In [44]:
gened_areas = ['EC', 'MQR', 'LPS', 'WCGI', 'USED', 'IS', 'CE', 'SW' ,'Lib Art', 'Adv Lib Art', 'ID', 'Major_Core','Major_Elec']
course_gened_fulfillment_list = []

In [45]:
gened_fulfillment_map = {}
for core_type in ['Required Core', 'Flexible Core']:
    for course_info in gened_data['gened'][core_type]:
        areas = course_info.get('Area(s)', [])
        gened_fulfillment_map[course_info['Course']] = areas

In [46]:
#set of course in the core and electives for followup processing
major_core = set([course['Course'] for course in major_data[selected_major]['Core']])
major_elective = set([course['Course'] for course in major_data[selected_major]['Electives']])
Adv_lib = set([course['Course'] for course in lib_data['Adv Lib Art']])
Lib_art = set([course['Course'] for course in lib_data['Lib Art']])

In [47]:
import re

course_gened_fulfillment_list = [] # Clear the list before populating it

for course in courses:
    fulfilled_geneds = gened_fulfillment_map.get(course, [])

    if not isinstance(fulfilled_geneds, list):
            fulfilled_geneds = [fulfilled_geneds] if fulfilled_geneds else []

    # Check for major core and elective fulfillment
    if course in major_core:
        fulfilled_geneds.append('Major_Core')
    if course in major_elective:
        fulfilled_geneds.append('Major_Elec')

    # Check for Liberal Arts fulfillment
    if course in Lib_art:
        fulfilled_geneds.append('Lib Art')
    if course in Adv_lib:
        fulfilled_geneds.append('Lib Art')
        fulfilled_geneds.append('Adv Lib Art')

    # Check for ID fulfillment
    if course in ID_data or re.search(r'\d+ID$', course):
        fulfilled_geneds.append('ID')


    course_gened_fulfillment_list.append({
        'Courses in Minor': course,
        'Fulfilled Areas': fulfilled_geneds
    })

In [52]:
import pandas as pd

course_fulfillment_list_for_df = []
gened_areas_for_columns = gened_areas

for course_info in course_gened_fulfillment_list:
    course_name = course_info['Courses in Minor']
    fulfilled_areas = course_info['Fulfilled Areas']
    course_dict = {'Courses in Minor': course_name}
    for gened_abbr in gened_areas_for_columns:
        course_dict[gened_abbr] = 'X' if gened_abbr in fulfilled_areas else ''
    course_fulfillment_list_for_df.append(course_dict)

course_fulfillment_df = pd.DataFrame(course_fulfillment_list_for_df)
course_fulfillment_df = course_fulfillment_df.fillna('')

# Set the index to start from 1
course_fulfillment_df.index = range(1, len(course_fulfillment_df) + 1)

# Display the DataFrame with dotted horizontal and vertical lines and dynamic caption
display(
    course_fulfillment_df.style
    .set_properties(**{'border': '1px dotted black'})
    .set_caption(f'{selected_major} Major and {selected_minor} Minor Course Fulfillment Overview')
    .set_table_styles([{'selector': 'caption', 'props': [('font-size', '20px')]}]
    )
)

,Courses in Minor,EC,MQR,LPS,WCGI,USED,IS,CE,SW,Lib Art,Adv Lib Art,ID,Major_Core,Major_Elec
1,ENG 1141,,,,,,,X,,X,,,,
2,ENG 2142,,,,,,,,,X,X,,,
3,ENG 2143,,,,,,,,,,,,,
4,ENG 2144,,,,,,,,,X,X,,,
5,ENG 2145,,,,,,,,,X,X,,,
6,ENG 1151,,,,,,X,,,X,,,,
7,ENG 2001,,,,,,,X,,X,X,,,
8,ENG 2002,,,,,,,X,,X,X,,,
9,ENG 2003,,,,,,,X,,X,X,,,
10,ENG 2420,,,,,,X,,,X,X,,,


In [15]:
import pandas as pd
# Assuming course_fulfillment_df is already created

full_df = course_fulfillment_df.copy()

# 1. Separate the 'Course' column and drop it from the data columns
course_names_series = full_df['Course']
full_df = full_df.drop(columns=['Course'])

# 2. Prepare the Index DataFrame
old_idx = course_names_series.to_frame(name='Course Name')

# 3. Define the new top index level ('Index Group') based on the user's rules
courses_to_group_as_choose_1 = ['MAT 1630', 'CST 1201']

# Use a list comprehension to assign the index group based on course name
index_group_level = [
    'Choose 1' if course in courses_to_group_as_choose_1 else 'Choose'
    for course in course_names_series
]

# 4. Insert the new top index level
old_idx.insert(0, 'Index Group', index_group_level)

# 5. Create the final MultiIndex and assign it back to the DataFrame
full_df.index = pd.MultiIndex.from_frame(old_idx)

# Display the Final Result
# ... (Output below)

In [27]:
display(full_df.style.set_properties(**{'border': '1px dotted black'})
.set_caption(f'{selected_major} Major and {selected_minor} Minor Course Fulfillment Overview').set_table_styles([
    {'selector': 'caption', 'props': [('font-size', '20px')]}
]))